In [ ]:
import constants
import pandas  as pd
from tqdm import tqdm
from pandarallel import pandarallel
import numpy as np
import scipy.stats
import time
import itertools
import random
import os
import collections
import json
import pickle

In [ ]:
import importlib
importlib.reload(constants)

In [ ]:
zipcodes = ["23222 60523 17320 58202 98605 33710 34240 37025 59020 90290 44857 63026 28726 60004 61084 27587 99143 93226 87048 98235 11372 18930 28571 61032 11762 79511 57006 19096 14847 95966 80620 85746 93634 86038 68633 35749 36611 40440 17048 67480 85296 97911 17748 37115 08824 79848 06235 25827 07853 02669 12461 93446 52404 93626 65722 97498 22314 60620 37762 37010 59105 19066 98030 57232 57567 72751 76225 59442 42048 61319 22973".split(" "), "17047 31027 97635 21250 19054 56171 89825 32608 44667 15622 20110 20710 27944 90018 70533 65560 13327 39421 80021 74849 57035 50322 60171 92562 66208 33023 78957 02645 71411 66062 10154 35131 88210 59275 27502 30817 98110 81144 50276 50638 21930 79772 27242 14843 58005 64473 75061 30552 15832 44833 08848 03263 02562 14131 02809 15676 80736 67127 15412 56477 90623 31003 10475 57064 28365 34208 30442 19081 93402 04001 12086".split(" "), "05761 07011 73055 90240 38541 01089 89118 80107 15470 39574 14739 48835 60409 77005 21734 44870 19103 54013 45504 62056 08752 54411 17370 39440 17976 23947 50105 21045 11419 32829 14040 14143 37019 08830 51040 01460 17339 81047 75563 49920 73063 87556 93424 55025 51246 90021 67353 13618 72116 24925 16301 49733 37814 33020 15135 76301 71929 41086 35208 32779 81052 48122 32351 37213 06066 27921 64156 98822 74105 85298 82222".split(" ")]

In [ ]:
tqdm.pandas()
pandarallel.initialize()

# Recovery by recreating the strata and finding a match 

In [ ]:
df_population = pd.read_csv(constants.CENSUS_2019_ZCTA_POPULATION,sep=",",header=[1]).rename(columns=lambda x: x.strip())
df_gazetteer = pd.read_csv(constants.CENSUS_2019_GAZETTEER,sep="\t").rename(columns=lambda x: x.strip())
df_income = pd.read_csv(constants.CENSUS_2019_ZCTA_INCOME,sep=",",header=[1]).rename(columns=lambda x: x.strip()).rename(columns=lambda x: x.strip())

In [ ]:
df_population["ZCTA5"] = df_population["Geographic Area Name"].str.slice(-5)
df_income["ZCTA5"] = df_income["Geographic Area Name"].str.slice(-5)
df_gazetteer["ZCTA5"] = df_gazetteer["GEOID"].astype("str").str.zfill(5)

df_population = df_population.set_index("ZCTA5")
df_income = df_income.set_index("ZCTA5")
df_gazetteer = df_gazetteer.set_index("ZCTA5")

In [ ]:
df_population["median_income"] = df_income["Estimate!!Families!!Median income (dollars)"].str.replace("+","").str.replace(",","").map(lambda x: "0" if x == "-" or x is None else x).str.replace("-","").astype("int")

In [ ]:
df_population["median_income"]

In [ ]:
df_population["area"] = df_gazetteer.ALAND
df_population["density"] = df_population["Estimate!!Total"] / df_population.area

# Stratify

In [ ]:
n_strata = 5 #Number of strata for each study

In [ ]:
df_population = df_population.sort_values("density",ascending=False)
n_zips = len(df_population)

pop_count = df_population["Estimate!!Total"].sum()
print(pop_count)

strata_density = [None] * n_strata

strata_to_zips_density = collections.defaultdict(list)
zip_to_strata_density = collections.defaultdict(None)

stratum_start = 0
zip_loc = 0
for i in range(n_strata):
    print(i)
    zip_start = zip_loc
    stratum_end = stratum_start
    stratum_goal = int((i+1) * pop_count / n_strata)
    while stratum_end < stratum_goal:
        #Lookup tables
        zipcode = df_population.index[zip_loc]
        strata_to_zips_density[i].append(zipcode)
        zip_to_strata_density[zipcode] = i
        
        #Strata size
        stratum_end += df_population["Estimate!!Total"].iloc[zip_loc]
        zip_loc += 1
    print(f"""Population of stratum: {stratum_end - stratum_start} (range: {stratum_start} - {stratum_end}, end goal: {stratum_goal})
Density range: {df_population.density[zip_loc]} {df_population.density[zip_start]}
Number of zipcodes: {zip_loc-zip_start} ({zip_start} - {zip_loc})
          """)
    
    strata_density[i] = df_population.iloc[zip_start:zip_loc].sample(frac=1).index.array
    
    stratum_start = stratum_end

In [ ]:
df_population = df_population.sort_values("median_income",ascending=False)
n_zips = len(df_population)

pop_count = df_population["Estimate!!Total"].sum()
print(pop_count)


strata_to_zips_income = collections.defaultdict(list)
zip_to_strata_income = collections.defaultdict(None)

strata_income = [None] * n_strata

stratum_start = 0
zip_loc = 0
for i in range(n_strata):
    print(i)
    zip_start = zip_loc
    stratum_end = stratum_start
    stratum_goal = int((i+1) * pop_count / n_strata)
    while stratum_end < stratum_goal:
        
        #Lookup tables
        zipcode = df_population.index[zip_loc]
        strata_to_zips_income[i].append(zipcode)
        zip_to_strata_income[zipcode] = i
        
        #Strata size
        stratum_end += df_population["Estimate!!Total"].iloc[zip_loc]
        zip_loc += 1
    print(f"""Population of stratum: {stratum_end - stratum_start:,d} (range: {stratum_start:,d} - {stratum_end:,d}, end goal: {stratum_goal:,d})
Median income range: {df_population.median_income[zip_loc]:,d} {df_population.median_income[zip_start]:,d}
Number of zipcodes: {zip_loc-zip_start:,d} ({zip_start:,d} - {zip_loc:,d})
          """)
    
    strata_income[i] = df_population.iloc[zip_start:zip_loc].sample(frac=1).index.array
    
    stratum_start = stratum_end

In [ ]:
df_population.loc['10154']

In [ ]:
def load_businesses(zipcode):
    with open(f"../../data/yelpzip_businesses/{zipcode}.json") as fp:
        return len(json.load(fp))

In [ ]:
businesses_by_zipcode = {zc: load_businesses(zc) for zc in itertools.chain.from_iterable(zipcodes)}

In [ ]:
businesses_by_zipcode

In [ ]:
df_strata = pd.DataFrame({zc:[zip_to_strata_density.get(zc,None),zip_to_strata_income.get(zc,None),businesses_by_zipcode.get(zc,None),df_population["Estimate!!Total"].loc[zc]] for zc in itertools.chain.from_iterable(zipcodes)}).T.rename({0:"Density",1:"Income",2:"Business Count",3:"Population"},axis=1)

In [ ]:
#Probably means the value was so low it got cut off?
df_strata = df_strata.fillna(4)

In [ ]:
df_strata[df_strata.Income == 4]["Business Count"].sum()

In [ ]:
for zcl in zipcodes:
    for zc in zcl:
        print(f"{zc}\t{zip_to_strata_density.get(zc,None)}\t{zip_to_strata_income.get(zc,None)}\t{businesses_by_zipcode.get(zc,None)}")
    print("-------------------------------")

In [ ]:
full_zip_list = list(itertools.chain.from_iterable(zipcodes))

NUM_STRATA = 5
#POP_GOAL = 100000
BUSINESS_GOAL = 500

def make_attempt(seed=None):
    if seed is None:
        raise Exception("Seed cannot be None")
    #Shuffle the zipcodes
    rand = random.Random(seed)
    shuffled_zipcodes = rand.sample(full_zip_list, k=len(full_zip_list))
    idx = 0
    
    #Greedily add to density, then income
    #Zipcode list for each strata
    density_strata = [[] for strata_num in range(NUM_STRATA)]
    income_strata = [[] for strata_num in range(NUM_STRATA)]
    
    #Business and population count for each stratum
    density_strata_businesses = [0 for strata_num in range(NUM_STRATA)]
    income_strata_businesses = [0 for strata_num in range(NUM_STRATA)]
    
    #Is the stratum sample filled?
    density_strata_status = [False for strata_num in range(NUM_STRATA)]
    income_strata_status = [False for strata_num in range(NUM_STRATA)]
    
    for zc in shuffled_zipcodes:
        row = df_strata.loc[zc]
        
        density_stratum = int(row.Density)
        income_stratum = int(row.Income)
        
        #Try to assign it to density, if not assign it to income
        if not density_strata_status[density_stratum]:
            density_strata[density_stratum].append(zc)
            density_strata_businesses[density_stratum] += row["Business Count"]
            
            #We hit a stratum goal
            if density_strata_businesses[density_stratum] > BUSINESS_GOAL:
                density_strata_status[density_stratum] = True
        elif not income_strata_status[income_stratum]:
            income_strata[income_stratum].append(zc)
            income_strata_businesses[income_stratum] += row["Business Count"]
            
            #We hit a stratum goal
            if income_strata_businesses[income_stratum] > BUSINESS_GOAL:
                income_strata_status[income_stratum] = True
        else:
            #Whoops, looks like we have nowhere to put this zipcode. Try again
            return False
    return density_strata, income_strata

In [ ]:
hits = []

seed = 1
while seed < 10000000:
    if seed % 1000 == 0:
        print(f'Attempt {seed}', end='\r')
    res = make_attempt(seed=seed)
    if res is not False:
        density_strata, income_strata = res
        print(f"Success!\nSeed: {seed}\nDensity strata:{density_strata}\nIncome strata:{income_strata}")
        hits.append((seed,res))
        break #Remove this to get more hits
    seed += 1

In [ ]:
hit_sets = set([
    tuple([
        tuple([
            tuple(sorted(stratum)) for stratum in strata
    ]) for strata in strata_list]) for seed, strata_list in hits]
)

In [ ]:
len(hit_sets)

In [ ]:
hit_sets

In [ ]:
zc_to_strata = {}
seed, (density_strata, income_strata) = hits[0] #Just take the first one, they're all equally good, right?
for i in range(len(density_strata)):
    for zc in density_strata[i]:
        zc_to_strata[zc] = (constants.CrawlExperiment.DENSITY, i)

for i in range(len(income_strata)):
    for zc in income_strata[i]:
        zc_to_strata[zc] = (constants.CrawlExperiment.INCOME, i)

In [ ]:
with open(constants.ZIPCODE_TO_STRATA, "wb+") as fp:
    pickle.dump(zc_to_strata,fp)

In [ ]:
with open(constants.CENSUS_STRATA_DATA, "wb+") as fp:
    df_strata.to_pickle(fp)

# Based on the git repo

In [ ]:
zipcodes = list(itertools.chain.from_iterable(zipcodes))

In [ ]:
density_strata = {
0: """11419 08830 11762 60409 90018 33020 07011 20390 19103 02721 92617 77014 33142 33322 95822 53214 85201 44121 48146 94544 10303 77081 21202 90001 11218 38677 79412 07628 60201 10459 85013 46224 64105 77551 19082 07505 63130 00909 07646 21206 97006 53233 55409 75231 77401 90262 19132 40203 60016 19128 94301 90240 92410 20783 33071 63130 14604 55405 20390 48825 97030 20032 80003 92867 53202 12308 55414 33183 11514 41099 60415 29409 33603 91205 75287 95128 32603 33064 78741 21234 60656 94402 92335 91367 10017 90680 00920 28533 90041 33615 19147 10282 30363 77062 14212 19125 11365 06269 77006 98115 19601 11423 91325 00901 06106 11590 11221 84102 07502 05401 33134 33126 33183 13902 99204 75243 94087 33150 19106 65215 20019 48126 80231 60649 19027 95843 44243 16228 80922 07062 91030 55403 60707 95051 60641 92806 58076 94303 19120 80231 10027 11001 91723 17550 17104 91722 94608 80004 92782 40508 61801 80218 77055 38505 19074 71272 93202 78229 00957 33185 10279 14607 80310 77019 22904 44108 02723 77007 19148 95820 02127 33139 90241 11205 85013 02912 14222 91776 11209 84120 17507 18225 48220 90048 33127 11220 07020 66019 90717 60016 28203 33136 92801 10927 28212 93906 14853 90061 50011 89103 78201 71209 22311 70118 94579 94116 33193 95351 84106 30306""".split(" "),
1: """19054 19066 17749 19081 34208 74105 85296 21045 80620 60171 48122 46998 35208 75061 21734 19096 46254 64063 21040 30043 19038 13116 30311 48026 55128 08869 78539 60466 92078 11725 63123 44130 28204 64052 43002 19046 91360 07036 85297 37013 30305 39531 76137 99202 83840 13410 78759 77518 77018 94509 94509 48084 46613 83706 98166 77073 95661 08109 64052 68154 14204 79904 74128 85714 94538 66202 43204 06514 40207 94305 11782 72801 14204 94925 79930 60561 06517 66207 60438 23225 91344 20165 75210 63123 37208 28207 19118 97217 08401 46222 17113 91789 85234 98106 53714 80227 95382 98155 12047 95677 19804 06108 30002 22151 80005 60406 68114 97007 43609 44140 11732 77018 94572 92835 33413 63053 33156 44142 32208 20817 23606 95486 13214 77379 45240 87114 50324 53235 60517 06401 46220 60162 48120 33407 85392 08824 53209 94542 11729 70058 15229 70131 19072 85345 33315 46107 93722 60478 60123 38119 33067 43068 92503 23229 92118 44128 98033 98008 43623 85351 55428 43608 95628 12304 55076 38135 80031 94130 73107 77082 84117 44710 33609 85295 64118 37909 75229 01074 43604 90049 12305 32208 46802 91326 00728 74128 55427 60432 75235 32211 14206 79701 98058 19453 08007 41074 15145 24016 34695 38114 48084 97229 33156 02026 02876 73162 80916 75043 06870 07750""".split(" "),
2: """32608 47335 15135 63026 89118 06066 98110 28726 66062 31410 01376 00616 78503 39440 32530 32963 18020 19422 07719 60103 92584 45415 43157 18518 10598 73149 70820 44653 20759 32901 15027 01438 17025 48327 18641 95431 23831 03054 01570 43537 08520 33605 45330 37217 68110 48168 10594 48324 15342 15739 08073 40272 97302 44053 34242 70811 29715 19967 34243 77385 44907 48607 33330 78596 15237 76102 18085 33811 98433 46311 48047 94027 35119 62034 62205 48045 15739 13157 73108 33983 18062 39530 00778 85259 50320 71461 33909 72204 33954 34474 05403 33812 45417 37343 48025 04240 27607 30327 30127 45740 63376 64503 15691 13064 12803 35401 83705 06089 44515 63042 18969 43974 06109 72301 94545 08251 30045 08810 23692 21701 21641 61833 83605 85353 27545 23112 10992 44672 33896 30094 78594 61111 84045 10598 80023 62018 08060 43961 18104 03826 08619 70769 46516 66614 08341 33647 15321 76179 34731 33170 44212 78574 75002 33803 41101 19454 38133 30094 92583 71111 49085 25040 94505 37130 28117 44484 01001 17070 30284 29607 39564 70047 32927 63134 06332 15739 79920 13760 01826 73170 44509 68123 33498 38125 83605 43525 77627 23692 47263 41526 15631 85120 47280 34268 06074 84037 07980 28428 07738 28480 10901 01040 20794 97218 01754 76155 39534 17865 62090 35758""".split(" "),
3: """37762 55025 17748 40440 64156 72751 85746 61032 92562 27587 90290 12432 12086 15922 08752 44870 21717 35126 46034 21795 14592 02346 08833 47855 17075 78730 44444 15532 21769 04430 05446 64507 17979 17505 32766 19946 04039 26586 93215 17701 44493 23651 80138 98595 21102 35611 10917 46146 30252 78666 06351 60417 43458 93605 30531 24503 14061 11963 78597 24701 21523 63070 27358 17067 39525 79606 04072 66046 44074 85138 85755 01092 61071 63012 44678 34797 12722 35950 30423 16102 35126 29626 31324 10926 22508 19522 16244 85635 62811 48381 12542 08853 28007 27258 49022 12789 15863 28350 37394 15736 21013 31627 61346 01451 17979 49863 19564 17301 46140 10536 43547 61554 19530 20777 34450 40059 78626 51656 93422 41030 15064 61332 01094 62093 26323 70514 46738 36265 18427 46968 31558 77441 25311 31747 02804 00650 44050 12533 18360 63140 44231 83116 12839 20674 06478 97365 37806 70343 55811 32226 46540 17552 25419 95312 15438 32332 88345 95946 93646 22060 22066 45252 13845 13126 14756 03045 13601 47374 12507 15865 31098 44026 11719 12489 18612 20762 70706 14127 80132 50035 68133 02556 83686 11948 72007 13036 71426 26369 66105 76226 40854 53090 38781 96779 64858 92264 59414 15126 02493 92823 01879 24503 72923 32509 48105 12722 70517 98290 38654 98333""".split(" "),
4: """88436 23884 57006 62688 63544 25875 81144 28365 06235 25169 27242 40868 36749 50638 82222 93446 31027 52755 48835 67353 02669 15470 14040 27921 74849 87556 42054 58005 81047 24984 41772 74521 38541 14843 65722 99922 86038 82221 15622 49920 36505 81052 98822 32351 38580 71411 14143 51040 30817 65560 31003 28571 50105 57035 82512 23947 30442 68381 42266 56517 12459 89825 17047 14847 57567 56477 12461 17320 98605 89815 62664 46746 73567 44867 84773 79739 56282 46926 54853 83601 16111 03256 29448 76528 96731 99701 24554 62357 62230 73659 59082 55367 71447 04758 93240 08041 64725 74571 13691 18240 04490 42028 63073 53553 28454 70532 82422 35121 14091 70755 81122 79713 37357 35078 75117 71823 74574 53549 64771 76652 28510 24323 95638 42167 24361 63344 67550 36751 38040 73018 59276 82070 56281 67059 05860 64720 36039 23146 12738 52729 93561 24236 72453 25875 58365 68018 95252 40350 71469 68879 55983 56057 14846 71358 72830 62447 48014 52040 36541 78375 30336 44889 80435 62467 15478 68701 70082 65735 78104 88344 54896 45773 62889 50626 67301 67140 57278 88418 45679 37731 05362 26651 32033 16645 36477 29567 05845 37680 05682 75127 57221 25234 37095 84737 47989 24472 13334 38052 62975 43152 84313 36034 58458 87413 80019 17920 54210 14009 26338 24528""".split(" ")
}


income_strata = {
0: """01460 60523 07853 58481 60004 08824 02562 32779 66208 83124 27502 11411 90623 94510 94619 95956 55347 02812 80107 15676 22625 22314 50226 08098 95014 77005 08848 58047 93424 43146 85298 94954 10519 03036 77316 20896 38017 77062 12309 95118 20036 93105 01583 55387 94019 55376 10524 06070 07950 90503 06762 07083 43566 98070 22911 02739 15238 94930 33149 98290 33558 53150 94619 02129 53202 28173 94020 58441 94703 23422 94706 15676 44139 62236 07030 05445 94952 68413 08202 93730 64156 07981 58580 07027 22124 93962 30149 46032 49116 20036 01036 44286 94930 18915 57016 77056 55902 05084 03032 02822 03609 06795 31522 20861 02420 03087 46845 55906 08751 51343 03861 94901 01740 10506 78663 07059 20132 50252 06423 94925 90069 45434 38947 76070 95136 45174 12147 84095 90278 89411 06107 18929 91326 02025 36064 07009 11975 98391 78681 93021 90035 72223 01566 85310 01532 06783 22046 55375 21612 21738 37408 98006 37219 47708 94306 48363 37067 75204 97209 94502 58007 20606 11516 90623 80919 60043 30621 10994 21718 21044 07723 07043 78108 07876 77389 03079 19003 01267 63025 80202 85297 85085 49406 10017 02561 02461 02492 23651 19806 85253 08056 21758 11777 98034 75230 10580 78751 55042 55410 07068 78663 92887 35763 06069 58768 61725 91360 55417 18078 90274""".split(" "),
1: """35749 93626 02645 50276 95251 80021 54013 34240 37010 44678 17370 99023 65054 75573 50322 17339 06247 65760 57064 93402 13082 70733 95703 18930 61863 20110 57232 87048 02809 90240 05356 66618 42054 72116 30107 84660 83112 99826 91016 56549 79325 65279 57449 37174 37085 35802 40046 60545 98338 98125 98125 27455 49417 61335 83466 53182 65076 44136 23060 98372 84106 05487 49686 03819 84050 80644 37363 62097 50863 66842 60090 52101 89123 08088 17968 92398 58770 45153 18415 51445 46234 28679 46341 90638 49902 03255 99565 57644 35749 18034 91941 03782 21231 19951 58746 08071 73165 66021 92110 56734 99029 12440 44212 46180 58227 60053 12760 58063 68003 92626 50035 08752 36113 53020 74053 74114 73162 14026 86023 83533 58210 15710 46250 35124 21078 15046 04917 51510 61243 18241 97070 07077 77566 56139 48197 33618 19070 92008 55445 68462 54025 60421 75415 64438 68715 84325 83127 58212 37122 12884 30092 49283 46182 56078 93426 10465 45011 80536 67521 64015 73056 70774 76028 46064 21402 56377 60707 02919 60453 85340 47968 18441 59218 93444 23149 37118 44319 62375 40242 52773 68357 58788 48823 97408 97007 15710 99624 52749 78006 03819 55381 95361 55749 56231 58278 58071 01056 95062 55074 77339 06052 89129 84087 29355 61273 23236 30813 46142 70778 46068""".split(" "),
2: """54411 52404 28626 67127 04001 56171 68633 33710 76225 58325 80736 05761 24847 12873 79511 14131 01084 51246 03263 13618 73063 01089 88210 61084 59105 98030 78957 32829 45504 78723 23805 52761 19405 44085 98257 12823 60476 56214 32063 61919 62883 56684 50428 37774 62273 98588 49236 98267 57648 18834 52777 51357 35811 17756 54750 60536 57345 68437 30294 61421 79372 24211 14437 68980 47386 50554 84053 32658 79502 06810 57036 32174 61378 82214 56629 50036 61030 05866 04274 27858 91931 14020 32757 51443 48638 54440 92683 50102 44640 54140 57223 47468 76569 98148 69301 78016 52236 76365 49446 47223 62088 45701 24015 28785 89128 28786 63755 75763 66214 66856 31303 84102 42031 54454 70711 68370 18443 76871 35490 27856 19934 18445 82501 68964 80017 50134 29075 54111 54740 61115 50028 62314 52584 61350 26038 35504 59105 44841 61721 26559 04239 14507 49712 05663 58622 03882 76460 44215 95519 68524 18704 64482 50162 52044 34957 48166 16029 53930 44460 19362 52584 50002 51521 66550 84101 12444 72087 89005 67736 23866 75411 56096 93532 89008 54981 45876 56716 20618 43732 04274 15650 64093 92879 02780 56041 60123 45385 61065 50134 78071 42039 18445 56473 67871 12592 15765 63347 30220 15077 31823 35094 57536 57268 15745 92313 17972 45013 21207 92806 47946""".split(" "),
3: """49891 49927 15412 43151 22973 16301 99143 73673 75563 67480 10475 20710 25827 37025 44857 66834 61319 39574 79772 23934 27944 59020 49026 76845 90021 47535 35063 64473 14739 95966 37019 59275 71929 59442 46911 41086 16834 70533 48885 04958 49733 13327 76845 62056 62461 17048 99012 73055 51639 44667 11372 61469 16657 57437 55332 72176 29564 43135 15683 57450 05647 78349 63934 24486 32694 65620 43772 83401 35183 42351 55787 97352 83655 28442 70578 13684 45164 24269 46991 84759 90021 38456 50123 38601 27967 73057 37642 48454 46805 47165 37921 69150 54408 84532 70466 29689 40051 98592 75792 68781 98564 54433 27521 17262 67123 32725 93101 47531 79377 73645 79772 68753 16230 31641 61414 97366 87531 23413 75706 45623 24944 48465 52639 30145 92802 41035 74462 15741 54970 84747 54534 15459 58524 90021 02838 39826 85623 50585 63030 91906 58056 71407 64434 70533 43157 28610 30734 80905 24326 87301 17211 17830 05079 29162 98832 14113 56297 19151 76134 24935 83686 83629 57355 78934 35183 12810 64841 45389 71720 59028 74881 17865 29911 67505 22940 56121 56248 16440 77357 92841 50071 40117 98649 62821 67345 66857 85335 27332 82839 25918 69154 95374 65567 15224 76679 26301 35772 80653 67468 85204 68710 35672 47265 05149 93440 82712 84770 49504 14513 65350""".split(" "),
4: """72670 98235 76841 58202 76301 37814 17976 33023 97635 68749 21250 70753 15784 93634 48551 39421 32190 97911 24925 39440 15832 93226 21930 26611 36611 30552 79848 70662 10154 37115 42048 37213 23222 44833 35131 97498 60620 72555 74830 35574 32420 38070 89427 71286 48737 50703 72351 63870 97442 36723 29178 72108 74901 45205 38077 23123 21902 79930 28433 21781 35443 74727 65601 22035 30238 47382 16637 82221 49098 59864 48449 45746 44243 83118 15764 79845 28508 41132 57339 38310 99759 78332 37315 81221 62017 43964 99167 97410 21215 60419 10177 28429 23958 71417 35477 69041 16876 04910 30575 78829 78516 26534 71243 22948 95249 67743 14608 25090 05439 40489 48227 38828 16672 65215 31629 39483 83322 67492 13636 37755 27530 64050 93932 86042 41052 25674 58335 65611 77453 49665 33960 29918 74946 57543 79258 24201 04276 33542 93513 74365 70062 31805 93249 49971 49793 54465 40376 84766 17723 10953 39323 55402 21627 16334 17099 02872 87537 37753 01103 29556 33513 62961 42210 56186 23952 26238 57346 68960 67213 19477 47003 29069 70515 19523 41817 31305 04606 44454 32228 12194 36569 78144 99733 33130 88041 98207 80477 15443 32072 30457 44670 14756 63901 39146 16563 41503 18457 83636 23106 62967 40983 15731 31524 87001 47924 38606 75933 29617 38868 06114""".split(" ")
}

In [ ]:
zc_reverse = {}
doubled_zipcodes = {}
for stratum in density_strata:
    for i,zc in enumerate(density_strata[stratum]):
        if zc in zc_reverse:
            experiment2, stratum2, idx = zc_reverse[zc]
            if experiment2 == constants.CrawlExperiment.DENSITY and stratum2 == stratum:
                continue
            print(f"Warning: doubled zipcode: {zc}")
            doubled_zipcodes[zc] = (zc_reverse[zc],(constants.CrawlExperiment.DENSITY,stratum,i))
            print(f"{zc_reverse[zc]}\n{(constants.CrawlExperiment.DENSITY,stratum,i)}")
        zc_reverse[zc] = (constants.CrawlExperiment.DENSITY,stratum,i)
        

for stratum in income_strata:
    for i,zc in enumerate(income_strata[stratum]):
        if zc in zc_reverse:
            experiment2, stratum2, idx = zc_reverse[zc]
            if experiment2 == constants.CrawlExperiment.INCOME and stratum2 == stratum:
                continue
            print(f"Warning: doubled zipcode: {zc}")
            doubled_zipcodes[zc] = (zc_reverse[zc],(constants.CrawlExperiment.INCOME,stratum,i))
            print(f"{zc_reverse[zc]}\n{(constants.CrawlExperiment.INCOME,stratum,i)}")
        zc_reverse[zc] = (constants.CrawlExperiment.INCOME,stratum,i)

In [ ]:
for zc in zipcodes:
    if zc in doubled_zipcodes:
        print(f"Warning: experiment zipcode was doubled {zc}: {doubled_zipcodes[zc]}")
    if zc not in zc_reverse:
        print(f"Warning: zipcode missing {zc}")

In [ ]:
strata_max = {}
for zc in zipcodes:
    if zc in doubled_zipcodes:
        continue
    experiment, strata, idx = zc_reverse[zc]
    strata_max[(experiment,strata)] = max(strata_max.get((experiment,strata),-1),idx)

In [ ]:
strata_max

In [ ]:
#Manual assignments
zc_reverse["08824"] = (3, 0, 5)
zc_reverse["90240"] = (3, 1, 29)
zc_reverse["08752"] = (2, 3, 14)
zc_reverse["39440"] = (3, 4, 19)
zc_reverse["64156"] = (2, 3, 4)

In [ ]:
zc_to_strata = {}
for k in zc_reverse:
    zc_to_strata[k] = tuple(zc_reverse[k][:2])

In [ ]:
with open(constants.ZIPCODE_TO_STRATA, "wb+") as fp:
    pickle.dump(zc_to_strata,fp)